In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sys import path
%matplotlib inline

In [3]:
path.insert(0, '../src')
from setup import data_setup
from data_generator import DataGenerator
from model_trainer import ModelTrainer

In [4]:
dg = data_setup()
norm_data = dg.df()

In [68]:
def get_frags(loc='../data/FragLibData32_Converted2010 (1)/Fragment Table.csv'):
    df = pd.read_csv(loc)
    a = list(df.columns[0:])
    a[0] = float(a[0])
    a[-1] = int(a[-1])
    a[-2] = a[-2][0]
    df.columns = ['FragmentMass', 'FragmentLabel', 'Isotopes', 'Formula', 'FragmentID']
    b = {'FragmentMass': a[0], 'FragmentLabel': a[1], 'Isotopes':a[2] , 'Formula':a[3],'FragmentID':a[4]}
    df = pd.concat([pd.DataFrame(b, index=[0]), df], sort=False)
    return df

In [69]:
frags = get_frags()

In [92]:
def get_ranges(frags, length, max = 235):
    '''
    Computes no mans land spectras.
    '''
    ranges = [[x, x + 1] for x in range(length)]
    for mass in frags:
        i = int(mass)
        if mass < 235.043933:
            if round(mass) == i + 1 and mass < ranges[i][1]:
                ranges[i][1] = mass
            elif round(mass) == i and mass > ranges[i][0]:
                ranges[i][0] = mass
        else:
            ranges[i][0] = mass
            ranges[i][1] = i + .9871
    return ranges

In [93]:
from data_transformation import get_hydrocarbs
masses = list(frags['FragmentMass']) + get_hydrocarbs(51)

In [95]:
ranges = get_ranges(masses, 1000)

In [96]:
from data_transformation import generate_data
# 10,000 Examples
erred = generate_data(norm_data, 2, 2, True, [0, 0, 0])
for _ in range(10):
    erred = pd.concat([erred, generate_data(norm_data, 2, 2, True, [.3, .6, 1], True)], axis=0)
erred['target'] = erred['target'].apply(lambda a: a - 1 if a > 0 else a)
dg.set_df(erred)
erred = dg.calibrated_df()

In [ ]:
def get_spectra(masses, intensities, nom_masses_low, nom_masses_high):
    spectra = [0 for x in range(2000)]
    spectra_intensities = [0 for x in range(2000)]
    for i, mass in enumerate(masses):
        j = round(mass)
        num = mass - nom_masses_low[j]
        spectra_intensities[j] = intensities[i]
        if num < 0:
            spectra[j] = num
        else:
            spectra[j] = mass - nom_masses_high[j]
    return spectra, spectra_intensities